<a href="https://colab.research.google.com/github/steinhaug/stable-diffusion/blob/main/KISTA_AUTOMATIC1111_WebUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stable Diffusion - AUTOMATIC1111 WebUI - v1.0 ALFA

This colab is maintained by Steinhaug, purpose is having a quick way of starting up the Gradio WebUI while using saved settings for easy access.

#  System checks

In [ ]:
#@title System check - Verify that we are running with a GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
#@title System check - Check memory
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

# INSTALL

In [4]:
#@title Connect: Google Drive mount, enable saving of settings
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
#@title Load and verify settings - click run
from pathlib import Path
from IPython.display import display, Markdown
from ipywidgets import widgets, GridspecLayout, Layout
from configparser import ConfigParser

inip = Path("/content/drive/MyDrive")
if not inip.is_dir():
    from google.colab import drive
    drive.mount('/content/drive')
else:
    print("Drive already mounted, continue.");

INI_FILE_LOC = "/content/drive/MyDrive/sd-colab.ini"

def strtobool (val):
    """Quick cast tro boolean
    """
    val = val.lower()
    if val in ('y', 'yes', 't', 'true', 'on', '1'):
        return True
    elif val in ('', 'n', 'no', 'f', 'false', 'off', '0'):
        return False
    else:
        raise ValueError("invalid truth value %r" % (val,))

def getValue (savedconf, key, type):
    try:
        val = savedconf[key]
    except KeyError:
        val = ""
    if type == 'bool':
        return strtobool(val)
    else:
        return val

ext_controlnet = widgets.Checkbox( value=False, description='ControlNet and Open Pose', disabled=False, indent=False )
ext_civitai = widgets.Checkbox( value=False, description='Image and civitai', disabled=False, indent=False )
ext_huggingface = widgets.Checkbox( value=False, description='Huggingface weui', disabled=False, indent=False )

ext_inspiration = widgets.Checkbox( value=False, description='Inspiration', disabled=False, indent=False )
ext_artists_to_study = widgets.Checkbox( value=False, description='Artists to study', disabled=False, indent=False )
ext_StylePile = widgets.Checkbox( value=False, description='StylePile', disabled=False, indent=False )

modat_controlnet = widgets.Checkbox( value=False, description='ControlNet', disabled=False, indent=False )
modat_inspiration = widgets.Checkbox( value=False, description='Inspiration imgs', disabled=False, indent=False )

path_cpkt = widgets.Text( value="", placeholder='Path to main checkpoint file', disabled=False )
dir_cpkt = widgets.Text( value="", placeholder='Path to checkpoints folder', disabled=False )
dir_controlnet = widgets.Text( value="", placeholder='Path to controlnet models folder', disabled=False )

ckpt_15 = widgets.Checkbox( value=False, description='v1.5', disabled=False, indent=False )
ckpt_15inpaint = widgets.Checkbox( value=False, description='v1.5 inpaint', disabled=False, indent=False )
ckpt_instruct_pix2pix = widgets.Checkbox( value=False, description='InstructPix2Pix', disabled=False, indent=False )
ckpt_21 = widgets.Checkbox( value=False, description='v2.1', disabled=False, indent=False )

def btn_continue_event_handler(btn_object):
    grid.close();
    print('You pressed {}'.format(btn_object.description))
    print('Creating .ini')
    config_object = ConfigParser()
    config_object["CONF"] = {
        "ext_controlnet": ext_controlnet.value,
        "ext_civitai": ext_civitai.value,
        "ext_huggingface": ext_huggingface.value,
        "ext_inspiration": ext_inspiration.value,
        "ext_artists_to_study": ext_artists_to_study.value,
        "ext_StylePile": ext_StylePile.value,
        "path_cpkt": path_cpkt.value,
        "dir_cpkt": dir_cpkt.value,
        "dir_controlnet": dir_controlnet.value,
        "ckpt_15": ckpt_15.value,
        "ckpt_15inpaint": ckpt_15inpaint.value,
        "ckpt_instruct_pix2pix": ckpt_instruct_pix2pix.value,
        "ckpt_21": ckpt_21.value,
        "modat_controlnet": modat_controlnet.value,
        "modat_inspiration": modat_inspiration.value,


    }
    with open(INI_FILE_LOC, 'w') as conf:
        config_object.write(conf)
    from IPython.display import clear_output; clear_output(); print('[1;32m Settings confirmed! ✓')
    print( ext_controlnet.value )

# - - - - - - 
btn_continue = widgets.Button(description='Save configuration and continue', button_style='info', layout=Layout(width='auto', height='auto'))
btn_continue.on_click(btn_continue_event_handler)

btn1 = widgets.Button(description='EXTENSIONS', layout=Layout(width='auto', height='auto'))
btn1.style.button_color='White'
btn1.style.font_weight='bold'
btn2 = widgets.Button(description='PATHS', layout=Layout(width='auto', height='auto'))
btn2.style.button_color='White'
btn2.style.font_weight='bold'
btn3 = widgets.Button(description='CHECKPOINTS', layout=Layout(width='auto', height='auto'))
btn3.style.button_color='White'
btn3.style.font_weight='bold'
btn4 = widgets.Button(description='MODELS', layout=Layout(width='auto', height='auto'))
btn4.style.button_color='White'
btn4.style.font_weight='bold'

grid = GridspecLayout(6, 5)

grid[0, 0] = btn1
grid[1, 0] = ext_controlnet
grid[2, 0] = ext_civitai
grid[3, 0] = ext_huggingface

grid[1, 1] = ext_inspiration
grid[2, 1] = ext_artists_to_study
grid[3, 1] = ext_StylePile

grid[0, 2] = btn3
grid[1, 2] = ckpt_15
grid[2, 2] = ckpt_15inpaint
grid[3, 2] = ckpt_instruct_pix2pix
grid[4, 2] = ckpt_21

grid[0, 3] = btn4
grid[1, 3] = modat_controlnet
grid[2, 3] = modat_inspiration

grid[0, 4] = btn2
grid[1, 4] = path_cpkt
grid[2, 4] = dir_cpkt
grid[3, 4] = dir_controlnet

grid[5, :] = btn_continue

#display(Markdown('**EXTENSIONS**'))
display(grid)

inip = Path(INI_FILE_LOC)
if inip.is_file():
    # Load values
    config_object = ConfigParser()
    config_object.read(INI_FILE_LOC)
    savedconf = config_object["CONF"]
  
    ext_controlnet.value = getValue(savedconf, 'ext_controlnet', 'bool')
    ext_civitai.value = getValue(savedconf, 'ext_civitai', 'bool')
    ext_huggingface.value = getValue(savedconf, 'ext_huggingface', 'bool')
    ext_inspiration.value = getValue(savedconf, 'ext_inspiration', 'bool')
    ext_artists_to_study.value = getValue(savedconf, 'ext_artists_to_study', 'bool')
    ext_StylePile.value = getValue(savedconf, 'ext_StylePile', 'bool')

    modat_controlnet.value = getValue(savedconf, 'modat_controlnet', 'bool')
    modat_inspiration.value = getValue(savedconf, 'modat_inspiration', 'bool')

    path_cpkt.value = getValue(savedconf, 'path_cpkt', 'string')
    dir_cpkt.value = getValue(savedconf, 'dir_cpkt', 'string')
    dir_controlnet.value = getValue(savedconf, 'dir_controlnet', 'string')
    ckpt_15.value = getValue(savedconf, 'ckpt_15', 'bool')
    ckpt_15inpaint.value = getValue(savedconf, 'ckpt_15inpaint', 'bool')
    ckpt_instruct_pix2pix.value = getValue(savedconf, 'ckpt_instruct_pix2pix', 'bool')
    ckpt_21.value = getValue(savedconf, 'ckpt_21', 'bool')

# https://tutswiki.com/read-write-config-files-in-python/


GridspecLayout(children=(Button(description='EXTENSIONS', layout=Layout(grid_area='widget001', height='auto', …

In [ ]:
#@title installer and runtime
skip_install = False #@param {type:"boolean"}
from IPython.display import clear_output;
%cd /content/


if not skip_install:
    %pip install -q accelerate transformers ftfy bitsandbytes==0.35.0 gradio natsort safetensors xformers
    !git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui

    if ext_controlnet.value:
        !git clone https://github.com/fkunn1326/openpose-editor /content/stable-diffusion-webui/extensions/openpose-editor
        !git clone https://github.com/Mikubill/sd-webui-controlnet /content/stable-diffusion-webui/extensions/sd-webui-controlnet
    if modat_controlnet.value:
        !wget https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_openpose.pth -O /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models/control_sd15_openpose.pth
        !wget https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_depth.pth -O /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models/control_sd15_depth.pth
        !wget https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_scribble.pth -O /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models/control_sd15_scribble.pth

    if ext_civitai.value:
        !git clone https://github.com/camenduru/sd-civitai-browser /content/stable-diffusion-webui/extensions/sd-civitai-browser
        !git clone https://github.com/yfszzx/stable-diffusion-webui-images-browser /content/stable-diffusion-webui/extensions/stable-diffusion-webui-images-browser

    if ext_huggingface.value:
        !git clone https://github.com/camenduru/stable-diffusion-webui-huggingface /content/stable-diffusion-webui/extensions/stable-diffusion-webui-huggingface
        !pip install huggingface-hub==0.11.1

    if ext_inspiration.value:
        !git clone https://github.com/yfszzx/stable-diffusion-webui-inspiration /content/stable-diffusion-webui/extensions/stable-diffusion-webui-inspiration
    if modat_inspiration.value:
        !wget https://huggingface.co/datasets/yfszzx/inspiration/resolve/main/inspiration.zip -O /content/stable-diffusion-webui/extensions/stable-diffusion-webui-inspiration/inspiration.zip
        from zipfile import ZipFile
        with ZipFile("/content/stable-diffusion-webui/extensions/stable-diffusion-webui-inspiration/inspiration.zip", 'r') as zObject:
            zObject.extractall(path="/content/stable-diffusion-webui/extensions/stable-diffusion-webui-inspiration")
        !rm /content/stable-diffusion-webui/extensions/stable-diffusion-webui-inspiration/inspiration.zip

    if ext_artists_to_study.value:
        !git clone https://github.com/camenduru/stable-diffusion-webui-artists-to-study /content/stable-diffusion-webui/extensions/stable-diffusion-webui-artists-to-study

    if ext_StylePile.value:
        !git clone https://github.com/some9000/StylePile /content/stable-diffusion-webui/extensions/StylePile

    if ckpt_15.value:
        !wget https://huggingface.co/runwayml/stable-diffusion-v1-5/blob/main/v1-5-pruned.ckpt -O v1-5.ckpt
        !wget https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -O /content/stable-diffusion-webui/models/Stable-diffusion/v1-5.vae.pt
        #!wget https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -O /content/stable-diffusion-webui/models/Stable-diffusion/vae-ft-mse-840000-ema-pruned.ckpt

    if ckpt_15inpaint.value:
        !wget https://huggingface.co/runwayml/stable-diffusion-inpainting/resolve/main/sd-v1-5-inpainting.ckpt -O /content/stable-diffusion-webui/models/Stable-diffusion/v1-5-inpainting.ckpt

    if ckpt_instruct_pix2pix.value:
        !wget https://huggingface.co/timbrooks/instruct-pix2pix/resolve/main/instruct-pix2pix-00-22000.ckpt -O /content/stable-diffusion-webui/models/Stable-diffusion/instruct-pix2pix.ckpt

    if ckpt_21.value:
        !wget https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.ckpt -O /content/stable-diffusion-webui/models/Stable-diffusion/v2-1_768.ckpt
    print('[1;32mInstalling complete!')
else:
    print('[1;32mSkipping installers... done!')  


%cd /content/stable-diffusion-webui/
args = "--share --xformers --enable-insecure-extension-access"
if path_cpkt.value != "":
    args = args + ' --ckpt="' + path_cpkt.value + '"'
if dir_cpkt.value != "":
    args = args + ' --ckpt-dir="' + dir_cpkt.value + '"'

print('[1;32mLaunching WebUI')  
!python launch.py $args



In [ ]:
#@title Install AUTOMATIC1111 libs
%pip install -q accelerate transformers ftfy bitsandbytes==0.35.0 gradio natsort safetensors xformers
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui


## Extensions

In [ ]:
#@title Extensions: openpose-editor and controlnet
!git clone https://github.com/fkunn1326/openpose-editor /content/stable-diffusion-webui/extensions/openpose-editor
!git clone https://github.com/Mikubill/sd-webui-controlnet /content/stable-diffusion-webui/extensions/sd-webui-controlnet
#!wget https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_openpose.pth -O /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models/control_sd15_openpose.pth
#!wget https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_depth.pth -O /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models/control_sd15_depth.pth
#!wget https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_scribble.pth -O /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models/control_sd15_scribble.pth

from IPython.display import clear_output; clear_output(); print('[1;32mDone! ✓')

In [ ]:
#@title Extensions: Browsers, image and civitai
!git clone https://github.com/yfszzx/stable-diffusion-webui-images-browser /content/stable-diffusion-webui/extensions/stable-diffusion-webui-images-browser
!git clone https://github.com/camenduru/sd-civitai-browser /content/stable-diffusion-webui/extensions/sd-civitai-browser
from IPython.display import clear_output; clear_output(); print('[1;32mDone! ✓')

In [ ]:
#@title Extensions: StylePile
!git clone https://github.com/some9000/StylePile /content/stable-diffusion-webui/extensions/StylePile

In [ ]:
#@title Extensions: stable-diffusion-webui-artists-to-study
!git clone https://github.com/camenduru/stable-diffusion-webui-artists-to-study /content/stable-diffusion-webui/extensions/stable-diffusion-webui-artists-to-study

In [ ]:
#@title Extensions: stable-diffusion-webui-inspiration
!git clone https://github.com/yfszzx/stable-diffusion-webui-inspiration /content/stable-diffusion-webui/extensions/stable-diffusion-webui-inspiration
!wget https://huggingface.co/datasets/yfszzx/inspiration/resolve/main/inspiration.zip -O /content/stable-diffusion-webui/extensions/stable-diffusion-webui-inspiration/inspiration.zip
from zipfile import ZipFile
with ZipFile("/content/stable-diffusion-webui/extensions/stable-diffusion-webui-inspiration/inspiration.zip", 'r') as zObject:
    zObject.extractall(path="/content/stable-diffusion-webui/extensions/stable-diffusion-webui-inspiration")

In [ ]:
#@title Extensions: Huggingface weui
!git clone https://github.com/camenduru/stable-diffusion-webui-huggingface /content/stable-diffusion-webui/extensions/stable-diffusion-webui-huggingface
!pip install huggingface-hub==0.11.1

from IPython.display import clear_output; clear_output(); print('[1;32mDone! ✓')

## Checkpoints

In [ ]:
!wget https://huggingface.co/timbrooks/instruct-pix2pix/resolve/main/instruct-pix2pix-00-22000.ckpt -O instruct-pix2pix.ckpt
!wget https://huggingface.co/runwayml/stable-diffusion-inpainting/resolve/main/sd-v1-5-inpainting.ckpt -O v1-5-inpainting.ckpt

!wget https://huggingface.co/runwayml/stable-diffusion-v1-5/blob/main/v1-5-pruned.ckpt -O v1-5.ckpt
!wget https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -O v1-5.vae.pt

!wget https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -O vae-ft-mse-840000-ema-pruned.ckpt

!wget https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.ckpt -O v2-1_768.ckpt


In [ ]:
#@title Checkpoint: kimsteinhaug-3000 (already in ./MyDrive/models)
!cp /content/drive/MyDrive/trainstep_checkpoints/epoch=000001-step=000003000.ckpt /content/stable-diffusion-webui/models/Stable-diffusion/kimsteinhaug-3000.ckpt
from IPython.display import clear_output; clear_output(); print('[1;32mDone! ✓')

In [ ]:
#@title Checkpoint: Misc checkpoints...
#%cd /content/stable-diffusion-webui
#!wget https://huggingface.co/ckpt/sd15/resolve/main/v1-5-pruned-emaonly.ckpt -O /content/stable-diffusion-webui/models/Stable-diffusion/v1-5-pruned-emaonly.ckpt
!wget https://huggingface.co/ckpt/Protogen_V2.2/resolve/main/Protogen_V2.2.ckpt -O /content/stable-diffusion-webui/models/Stable-diffusion/Protogen_V2.2.ckpt
#!wget https://huggingface.co/ckpt/hassanblend1.4/resolve/main/HassanBlend1.4-Pruned.ckpt -O /content/stable-diffusion-webui/models/Stable-diffusion/HassanBlend1.4-Pruned.ckpt
from IPython.display import clear_output; clear_output(); print('[1;32mDone! ✓')

# Launch - Run it

In [ ]:
#@title Launcher: Default
%cd /content/stable-diffusion-webui/

# Launch WebUI
!python launch.py \
  --share \
  --xformers \
  --enable-insecure-extension-access \
  --ckpt="/content/drive/MyDrive/models/kimsteinhaug-3000.ckpt" \
  --ckpt-dir="/content/drive/MyDrive/models/"


In [ ]:
#@title Launcher: with skip install
#--update-check 
#--ui-settings-file=""
#share = "--share --api --cors-allow-origins=https://www.painthua.com --xformers --theme=dark --disable-safe-unpickle --enable-insecure-extension-access --no-half-vae" + " --ckpt=" + model_init.value + " --ckpt-dir=" + models_dir.value + " " + SDZ_Args.value

%cd /content/stable-diffusion-webui/
!python launch.py \
  --skip-install \
  --share \
  --xformers \
  --enable-insecure-extension-access \
  --ui-settings-file="/content/drive/MyDrive/models/config.json" \
  --ckpt="/content/stable-diffusion-webui/models/Stable-diffusion/kimsteinhaug-3000.ckpt" \
  --ckpt-dir="/content/drive/MyDrive/models/"

# Misc

In [ ]:
full body render of an alluring god, (kimsteinhaug:1.1)  as festival hippy with tribal tattoos surrounded by a underwater ink pour and flowing liquid gallium and sacred geometry, perfect body and face, sexy (kimsteinhaug:0.3), cinematic, beautifully lit, by miho hirano, by karol bak, by donato giancola, 3 d, trending on artstation, octane render, 8 k
lowres| worst quality| low quality| normal quality| signature| blurry| bad anatomy| bad hands| missing fingers| extra digit| fewer digits| cropped

/content/drive/MyDrive/models/
/content/drive/MyDrive/output/images/

https://github.com/JingyunLiang/SwinIR/releases
https://github.com/JingyunLiang/SwinIR
https://upscale.wiki/wiki/Model_Database

// 
To add: https://github.com/Extraltodeus/multi-subject-render
https://github.com/yownas/shift-attention


https://github.com/enlyth/sd-webui-riffusion

SyntaxError: ignored

In [ ]:
from ipywidgets import jslink, IntSlider
from ipywidgets import TwoByTwoLayout
from ipywidgets import HTML, AppLayout, Button, Layout


top_left_text = widgets.IntText(description='Top left', layout=Layout(width='auto', height='auto'))
top_right_text = widgets.IntText(description='Top right', layout=Layout(width='auto', height='auto'))
bottom_left_text = widgets.IntText(description='Top left', layout=Layout(width='auto', height='auto'))
bottom_right_text = widgets.IntText(description='Top right', layout=Layout(width='auto', height='auto'))

#app = TwoByTwoLayout( top_left=e1, top_right=e2, bottom_left=e3,  bottom_right=e4 )
#app.bottom_right.value = True
#app

#header = HTML("<h1>Select extensions to install</h1>", layout=Layout(height='auto'))
#header.style.text_align='center'
#footer = HTML("<h1>Fictional World Weather</h1>", layout=Layout(height='auto'))
#AppLayout(header=header,          left_sidebar=e1,          center=e2,          right_sidebar=e3,          footer=footer)


Button(description='Test', style=ButtonStyle())

You pressed the Test button!
You pressed the Test button!


In [ ]:

from configparser import ConfigParser
config_object = ConfigParser()
config_object["EXTENSIONS"] = {
    "controlnet": False,
    "civitai": False,
    "huggingface": False,
    "more": False
}

config_object["SERVERCONFIG"] = {
    "host": "tutswiki.com",
    "port": "8080",
    "ipaddr": "8.8.8.8"
}






a = widgets.Text(
    value="",
    placeholder='Path to main checkpoint file',
    description='String:',
    disabled=False
)
b = widgets.Text(
    value="",
    placeholder='Path to models directory',
    description='String:',
    disabled=False
)
c = widgets.Text(
    value="",
    placeholder='Another variable',
    description='String:',
    disabled=False
)
#display(var_ckpt)
#display(var_ckptdir)

def f(a, b, c):
    print('{}*{}*{}={}'.format(a, b, c, a+b+c))

out = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})

widgets.HBox([widgets.VBox([a, b, c]), out])

## Memory tools and others

In [ ]:
#@title Free up CPU-RAM
import gc

# Custom Callback To Include in Callbacks List At Training Time
class GarbageCollectorCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()

In [ ]:
#@title Free up GPU-RAM
torch.cuda.empty_cache()

In [ ]:
#@title Div patches...
# 2023-02 fix, in case this bug https://github.com/AUTOMATIC1111/stable-diffusion-webui/issues/7714
#!pip install --upgrade fastapi==0.90.1
